In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
# Load the data (assuming 'maize_df' is already loaded)
data = maize_df

# Set the 'date' column as the DateTimeIndex
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

# Downsample the data (e.g., to monthly frequency)
data = data.resample('M').mean()

# Plot the time series of maize prices per unit
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['price_per_unit'], label='Maize Price per Unit', color='blue')
plt.title('Time Series of Maize Prices per Unit')
plt.xlabel('Date')
plt.ylabel('Price per Unit')
plt.legend()
plt.grid(True)
plt.show()

# Check for stationarity using ADF test
adf_result = adfuller(data['price_per_unit'])
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])
print("Critical Values:", adf_result[4])

# If the data is not stationary, apply differencing
if adf_result[1] > 0.05:
    data['price_diff'] = data['price_per_unit'] - data['price_per_unit'].shift(1)

    # Plot differenced time series
    plt.figure(figsize=(12, 6))
    plt.plot(data.index[1:], data['price_diff'].iloc[1:], label='Differenced Price per Unit')
    plt.title('Differenced Time Series of Maize Prices per Unit')
    plt.xlabel('Date')
    plt.ylabel('Differenced Price per Unit')
    plt.legend()
    plt.show()

# Decompose the time series into trend, seasonal, and residual components
decomposition = seasonal_decompose(data['price_per_unit'], model='additive', period=12)  # Monthly seasonality

# Plot decomposed components
plt.figure(figsize=(12, 8))
plt.subplot(411)
plt.plot(data.index, decomposition.trend, label='Trend')
plt.title('Trend Component')
plt.subplot(412)
plt.plot(data.index, decomposition.seasonal, label='Seasonal')
plt.title('Seasonal Component')
plt.subplot(413)
plt.plot(data.index, decomposition.resid, label='Residual')
plt.title('Residual Component')
plt.tight_layout()

# Use ACF and PACF plots to determine the order of ARIMA model
plt.figure(figsize=(12, 6))
plot_acf(data['price_diff'].dropna(), lags=50, ax=plt.subplot(121))
plot_pacf(data['price_diff'].dropna(), lags=50, ax=plt.subplot(122))
plt.show()

# Train-test split
train_size = int(len(data) * 0.8)
train, test = data['price_per_unit'][:train_size], data['price_per_unit'][train_size:]

# Fit SARIMAX model
order = (1, 1, 1)  # Specify the ARIMA order (p, d, q)
seasonal_order = (1, 1, 1, 12)  # Monthly seasonal order (P, D, Q, S)
sarimax_model = SARIMAX(train, order=order, seasonal_order=seasonal_order)
sarimax_result = sarimax_model.fit()

# Model diagnostics
sarimax_result.plot_diagnostics(figsize=(12, 8))
plt.show()

# Forecast future prices
n_forecast = len(test)
forecast = sarimax_result.get_forecast(steps=n_forecast)

# Calculate MSE and RMSE
mse = mean_squared_error(test, forecast.predicted_mean)
rmse = sqrt(mse)
r_squared = r2_score(test, forecast.predicted_mean)
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R2): {r_squared}')